<a href="https://colab.research.google.com/github/ofekih/LightsOutChallenge/blob/main/LightsOutChallenge.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This will do $5+2$

In [1]:
5+2

7